# core

> A merry FastHTML nbdev project that uses Pygments for Christmas joy

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from execnb.nbio import *
from fastcore.all import *
from fasthtml.common import *
from fasthtml.components import show
from fasthtml.jupyter import *
from inspect import getsource
from IPython.display import display, HTML, IFrame
from pathlib import Path
import pygments
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from random import choice

## Get All Pygments Styles

In [ ]:
#| export
pygstyles = L(pygments.styles.get_all_styles())

In [ ]:
print(pygstyles)

['abap', 'algol', 'algol_nu', 'arduino', 'autumn', 'bw', 'borland', 'coffee', 'colorful', 'default', 'dracula', 'emacs', 'friendly_grayscale', 'friendly', 'fruity', 'github-dark', 'gruvbox-dark', 'gruvbox-light', 'igor', 'inkpot', 'lightbulb', 'lilypond', 'lovelace', 'manni', 'material', 'monokai', 'murphy', 'native', 'nord-darker', 'nord', 'one-dark', 'paraiso-dark', 'paraiso-light', 'pastie', 'perldoc', 'rainbow_dash', 'rrt', 'sas', 'solarized-dark', 'solarized-light', 'staroffice', 'stata-dark', 'stata-light', 'tango', 'trac', 'vim', 'vs', 'xcode', 'zenburn']


## From Python to Pygments-Highlighted FastTag

To display Python code in a FastHTML app with Pygments formatting, I made this FastTag:

In [ ]:
#| export
def PythonToPygmentsFT(c, style='rrt'):
    "Turns Python code into a Pygments syntax-highlighted FastTag"
    fm = HtmlFormatter(style=style, cssclass=style)
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".{style}")
    return Div(Style(sd), NotStr(h), id=style)

It includes the Pygments Python CSS for the style, scoped to a CSS class name that is the style name. That allows you to use it multiple times with different styles in the same page or notebook. See my [Scoped Syntax Highlighting](https://github.com/audreyfeldroy/arg-blog-fasthtml/blob/main/nbs/2024-12-27-Scoped-Syntax-Highlighting.ipynb) blog post for details.

### Examples

Here are a couple FTs created with it. To show a FT in a Jupyter notebook, I use `fasthtml.components.show`:

In [ ]:
pft = PythonToPygmentsFT('print("Hi Uma")')
show(pft)

In [ ]:
pft = PythonToPygmentsFT(getsource(PythonToPygmentsFT), style="monokai")
show(pft)

## Highlight a Selected Notebook Cell

Let's highlight a cell of this notebook.

In [ ]:
nb = read_nb(Path("00_core.ipynb"))
c = nb.cells[5]
c

```json
{ 'cell_type': 'code',
  'execution_count': None,
  'idx_': 5,
  'metadata': {},
  'outputs': [],
  'source': '#| export\npygstyles = L(pygments.styles.get_all_styles())'}
```

In [ ]:
show(PythonToPygmentsFT(c.source, style='dracula'))

## Get Code Cells of a Notebook

This section is all about throwing away non-code cells.

> Imagine those elves from Advent of Code doing whatever they want to a notebook

In [ ]:
#| export
def is_code_cell(c): return c.cell_type == 'code'

In [ ]:
L(nb.cells).filter(is_code_cell).itemgot('source')[:2]

(#2) ['#| default_exp core','#| hide\nfrom nbdev.showdoc import *']

In [ ]:
#| export
def get_code_cells(nb): return L(nb.cells).filter(is_code_cell).itemgot('source')

In [ ]:
get_code_cells(nb)

(#27) ['#| default_exp core','#| hide\nfrom nbdev.showdoc import *','#| export\nfrom execnb.nbio import *\nfrom fastcore.all import *\nfrom fasthtml.common import *\nfrom fasthtml.components import show\nfrom fasthtml.jupyter import *\nfrom inspect import getsource\nfrom IPython.display import display, HTML, IFrame\nfrom pathlib import Path\nimport pygments\nfrom pygments import highlight\nfrom pygments.lexers import PythonLexer\nfrom pygments.formatters import HtmlFormatter\nfrom random import choice','#| export\npygstyles = L(pygments.styles.get_all_styles())','print(pygstyles)','#| export\ndef PythonToPygmentsFT(c, style=\'rrt\'):\n    "Turns Python code into a Pygments syntax-highlighted FastTag"\n    fm = HtmlFormatter(style=style, cssclass=style)\n    h = highlight(c, PythonLexer(), fm)\n    sd = fm.get_style_defs(f".{style}")\n    return Div(Style(sd), NotStr(h), id=style)','pft = PythonToPygmentsFT(\'print("Hi Uma")\')\nshow(pft)','pft = PythonToPygmentsFT(getsource(PythonToPyg

## Highlight Code Cells of a Notebook

This is sort of like a toy version of nb2fasthtml's highly-customizable [render_nb](https://answerdotai.github.io/nb2fasthtml/core.html#render_nb) that throws away Markdown cells and code cell output:

In [ ]:
#| export
def NotebookToPygmentsFT(nb, style='rrt'):
    "Warning: This gets only code cells, without their output or any other cells"
    cells = get_code_cells(nb)
    return cells.map(partial(PythonToPygmentsFT, style=style))

To see what it does, uncomment this:

In [ ]:
# show(Div(*NotebookToPygmentsFT(nb)))

## Highlight Each Notebook Cell Randomly

In [ ]:
#| export
def PythonToRandomPygmentsFT(c,print_style=True): 
    style = choice(pygstyles)
    result = PythonToPygmentsFT(c, style=style)
    if print_style:
        result = Div(f"Pygments style: {style}", result)
    return result

In [ ]:
pft = PythonToRandomPygmentsFT('print("Hi Uma")', print_style=False)
show(pft)

In [ ]:
pft = PythonToRandomPygmentsFT('print("Hi Uma")', print_style=True)
show(pft)

In [ ]:
#| export
def NotebookToRandomPygmentsFT(nb):
    "Warning: This gets only code cells, without their output or any other cells"
    cells = get_code_cells(nb)
    return cells.map(PythonToRandomPygmentsFT)

## FastHTML App

In [ ]:
#| export
app,rt = fast_app(pico=False)

In [ ]:
server = JupyUvi(app)

### Routes and Handlers

In [ ]:
#| export
@rt
def index():
    "Random Pygments style for each cell"
    nb = read_nb(Path("../nbs/deck-the-halls.ipynb"))
    return (
        H1(f"Random Pygments Style For Each Cell"),
        P("This page gets a random ",
            A("Pygments", href="https://pygments.org/"),
            " HtmlFormatter style for each cell of ",
            A("my Deck the Halls Jupyter notebook", href="https://nbsanity.com/static/a426287f3fbfc5a38c99291beadc77d3/2024-12-24-deck-the-halls.html")),
        Div(*NotebookToRandomPygmentsFT(nb))
    )

In [ ]:
#| export
@rt
def randstyle():
    "Random Pygments style upon refresh"
    nb = read_nb(Path("../nbs/deck-the-halls.ipynb"))
    code_cells = get_code_cells(nb)
    style = choice(pygstyles)
    fm = HtmlFormatter(style=style)
    return (
        H1(f"Random style: {style}"),
        P("This page gets a random ",
            A("Pygments", href="https://pygments.org/"),
            " HtmlFormatter style and applies it to ",
            A("my Deck the Halls Jupyter notebook", href="https://nbsanity.com/static/a426287f3fbfc5a38c99291beadc77d3/2024-12-24-deck-the-halls.html")),
        Div(*NotebookToPygmentsFT(nb, style=style))
    )

## Stop Server

In [ ]:
if 'server' in globals(): server.stop()

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()